In [27]:
import pandas as pd
import numpy as np
import json

#First, we process the data by league. 


#This is the processing for epl

#First, we start with a dataframe to hold all seasons data
league_matches = pd.DataFrame({})

years = ['2024_2025', '2023_2024', '2022_2023', '2021_2022', '2020_2021', '2019_2020', '2018_2019', '2017_2018']

for year in years:
    test_df = pd.read_json(f'../season_data/game_data_serie_a_{year}.json')
    test_df['season'] = year
    league_matches = pd.concat([league_matches, test_df], ignore_index=True)

#Second, we create a numeric id for each team
league_matches['team_code'] = league_matches.home_team.astype('category').cat.codes

# print(league_matches.shape)

# print(league_matches.columns)

print(league_matches['round'].unique())
print(set(league_matches['home_team'].tolist() + league_matches['away_team'].tolist()))

#Check: Verify home teams and away teams are consistent
# print(sorted(league_matches.home_team.unique()) == sorted(league_matches.away_team.unique()))

#Now, I want to convert the wide data format to a long format by extracting each team's individual data froom each match week

#First, let us get the types of each column
#To get the types of each column, we use pd.info() 

league_matches.head()

['ROUND 21' 'ROUND 19' 'ROUND 20' 'ROUND 18' 'ROUND 17' 'ROUND 16'
 'ROUND 15' 'ROUND 14' 'ROUND 13' 'ROUND 12' 'ROUND 11' 'ROUND 10'
 'ROUND 9' 'ROUND 8' 'ROUND 7' 'ROUND 6' 'ROUND 5' 'ROUND 4' 'ROUND 3'
 'ROUND 2' 'ROUND 1' 'ROUND 29' 'ROUND 38' 'ROUND 37' 'ROUND 36'
 'ROUND 35' 'ROUND 34' 'ROUND 32' 'ROUND 33' 'ROUND 31' 'ROUND 30'
 'ROUND 28' 'ROUND 27' 'ROUND 26' 'ROUND 25' 'ROUND 24' 'ROUND 23'
 'ROUND 22' 'ADDITIONAL MATCH']
{'Parma', 'Empoli', 'Venezia', 'Cagliari', 'AC Milan', 'Genoa', 'Fiorentina', 'Lecce', 'Brescia', 'Chievo', 'Benevento', 'Sampdoria', 'Bologna', 'Torino', 'Juventus', 'Monza', 'Salernitana', 'Udinese', 'Como', 'Atalanta', 'Cremonese', 'Spal', 'Sassuolo', 'Frosinone', 'Inter', 'Lazio', 'Crotone', 'AS Roma', 'Spezia', 'Napoli', 'Verona'}


,comp,round,home_team,away_team,home_goals,away_goals,date,time,home_expected_goals_(xg),away_expected_goals_(xg),...,home_tackles,away_tackles,home_clearances_total,away_clearances_total,home_interceptions,away_interceptions,home_headed_goals,away_headed_goals,season,team_code
0,SERIE A,ROUND 21,Como,Udinese,4,1,2025-01-20,14:45,1.76,0.59,...,40% (6/15),68% (13/19),13.0,13.0,4.0,11.0,NaN,NaN,2024_2025,8
1,SERIE A,ROUND 21,Inter,Empoli,3,1,2025-01-19,14:45,1.58,0.27,...,55% (11/20),64% (9/14),9.0,28.0,7.0,15.0,1.0,0.0,2024_2025,15
2,SERIE A,ROUND 21,Verona,Lazio,0,3,2025-01-19,12:00,1.31,1.45,...,87% (13/15),60% (9/15),17.0,22.0,9.0,7.0,0.0,1.0,2024_2025,30
3,SERIE A,ROUND 21,Cagliari,Lecce,4,1,2025-01-19,09:00,2.74,0.50,...,67% (10/15),80% (8/10),12.0,21.0,7.0,5.0,1.0,0.0,2024_2025,6
4,SERIE A,ROUND 21,Parma,Venezia,1,1,2025-01-19,09:00,1.62,1.15,...,47% (8/17),74% (17/23),11.0,30.0,12.0,9.0,NaN,NaN,2024_2025,21


In [28]:
#First, let's convert the comp object to a string => comp stands for competition/league

#Ensuring no nan values
null_count = league_matches['comp'].isna().sum()
print(null_count)

league_matches['comp'] = league_matches['comp'].astype('str').str.lower().str.strip().str.replace(" ", "_")

league_matches.head()

0


,comp,round,home_team,away_team,home_goals,away_goals,date,time,home_expected_goals_(xg),away_expected_goals_(xg),...,home_tackles,away_tackles,home_clearances_total,away_clearances_total,home_interceptions,away_interceptions,home_headed_goals,away_headed_goals,season,team_code
0,serie_a,ROUND 21,Como,Udinese,4,1,2025-01-20,14:45,1.76,0.59,...,40% (6/15),68% (13/19),13.0,13.0,4.0,11.0,NaN,NaN,2024_2025,8
1,serie_a,ROUND 21,Inter,Empoli,3,1,2025-01-19,14:45,1.58,0.27,...,55% (11/20),64% (9/14),9.0,28.0,7.0,15.0,1.0,0.0,2024_2025,15
2,serie_a,ROUND 21,Verona,Lazio,0,3,2025-01-19,12:00,1.31,1.45,...,87% (13/15),60% (9/15),17.0,22.0,9.0,7.0,0.0,1.0,2024_2025,30
3,serie_a,ROUND 21,Cagliari,Lecce,4,1,2025-01-19,09:00,2.74,0.50,...,67% (10/15),80% (8/10),12.0,21.0,7.0,5.0,1.0,0.0,2024_2025,6
4,serie_a,ROUND 21,Parma,Venezia,1,1,2025-01-19,09:00,1.62,1.15,...,47% (8/17),74% (17/23),11.0,30.0,12.0,9.0,NaN,NaN,2024_2025,21


In [29]:
# Next, let us convert the round to an integer type

null_count = league_matches['round'].isna().sum()

print(null_count)

# in serie a, some rounds are played as additional match - so we need to convert these 
league_matches.loc[league_matches["round"] == "ADDITIONAL MATCH", 'round'] = "ROUND 39"

league_matches['round'] = league_matches['round'].astype('str').str.lower().str.split(" ", n=1).str[1]
null_count = league_matches['round'].isna().sum()
print(null_count)


# Now, we convert all rounds to integers
league_matches['round'] = league_matches['round'].astype('int')
# Print all unique rounds
print(league_matches['round'].unique())
league_matches.head()

0
0
[21 19 20 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1 29 38 37
 36 35 34 32 33 31 30 28 27 26 25 24 23 22 39]


,comp,round,home_team,away_team,home_goals,away_goals,date,time,home_expected_goals_(xg),away_expected_goals_(xg),...,home_tackles,away_tackles,home_clearances_total,away_clearances_total,home_interceptions,away_interceptions,home_headed_goals,away_headed_goals,season,team_code
0,serie_a,21,Como,Udinese,4,1,2025-01-20,14:45,1.76,0.59,...,40% (6/15),68% (13/19),13.0,13.0,4.0,11.0,NaN,NaN,2024_2025,8
1,serie_a,21,Inter,Empoli,3,1,2025-01-19,14:45,1.58,0.27,...,55% (11/20),64% (9/14),9.0,28.0,7.0,15.0,1.0,0.0,2024_2025,15
2,serie_a,21,Verona,Lazio,0,3,2025-01-19,12:00,1.31,1.45,...,87% (13/15),60% (9/15),17.0,22.0,9.0,7.0,0.0,1.0,2024_2025,30
3,serie_a,21,Cagliari,Lecce,4,1,2025-01-19,09:00,2.74,0.50,...,67% (10/15),80% (8/10),12.0,21.0,7.0,5.0,1.0,0.0,2024_2025,6
4,serie_a,21,Parma,Venezia,1,1,2025-01-19,09:00,1.62,1.15,...,47% (8/17),74% (17/23),11.0,30.0,12.0,9.0,NaN,NaN,2024_2025,21


In [31]:
# Next, let us process the home teams and away teams

league_matches['home_team'] = league_matches['home_team'].astype('str').str.lower().str.replace(' ', "_")
league_matches['away_team'] = league_matches['away_team'].astype('str').str.lower().str.replace(' ', "_")

# Ensuring no null values
print(league_matches['home_team'].isna().sum())
print(league_matches['away_team'].isna().sum())

# Next, let us get the unique home and away teams

all_teams = list(league_matches['home_team'].unique()) + list(league_matches['away_team'].unique())
all_teams = set(all_teams)


team_dict = {
    'ac_milan': "AC Milan",
    'as_roma': "AS Roma",
    'atalanta': "Atalanta BC",
    'benevento': "Benevento Calcio",
    'bologna': "Bologna FC",
    'brescia': "Brescia Calcio",
    'cagliari': "Cagliari Calcio",
    'chievo': "AC ChievoVerona",
    'como': "Como 1907",
    'cremonese': "US Cremonese",
    'crotone': "FC Crotone",
    'empoli': "Empoli FC",
    'fiorentina': "ACF Fiorentina",
    'frosinone': "Frosinone Calcio",
    'genoa': "Genoa CFC",
    'inter': "FC Internazionale Milano",
    'juventus': "Juventus FC",
    'lazio': "SS Lazio",
    'lecce': "US Lecce",
    'monza': "AC Monza",
    'napoli': "SSC Napoli",
    'parma': "Parma Calcio 1913",
    'salernitana': "US Salernitana 1919",
    'sampdoria': "UC Sampdoria",
    'sassuolo': "US Sassuolo Calcio",
    'spal': "SPAL",
    'spezia': "Spezia Calcio",
    'torino': "Torino FC",
    'udinese': "Udinese Calcio",
    'venezia': "Venezia FC",
    'verona': "Hellas Verona FC",
}

league_countries = {
    "bundesliga": "Germany",
    "champions_league": "Europe",  # International competition across Europe
    "europa_league": "Europe",     # International competition across Europe
    "laliga": "Spain",
    "ligue_1": "France",
    "mls": "United States",
    "premier_league": "England",
    "serie_a": "Italy",
    "usl": "United States"
}

# Ensuring no null values
print(league_matches['home_team'].isna().sum())
print(league_matches['away_team'].isna().sum())

# i notice some discrepancies in some team names. 
# for example, manchester united shows up as Manchester United and Manchester Utd
# so, let me standardize these names



# Verifying home and away teams are the same
sorted(league_matches['home_team'].unique()) == sorted(league_matches['away_team'].unique())

0
0
0
0


True

In [35]:
print(set(league_matches['home_team'].unique().tolist()+league_matches['away_team'].unique().tolist()))
print(len(set(league_matches['home_team'].unique().tolist()+league_matches['away_team'].unique().tolist())))
print(sorted(team_dict.keys()) == sorted(set(league_matches['home_team'].unique().tolist()+league_matches['away_team'].unique().tolist())))

{'as_roma', 'juventus', 'verona', 'lazio', 'benevento', 'bologna', 'udinese', 'torino', 'ac_milan', 'genoa', 'fiorentina', 'spal', 'cremonese', 'spezia', 'crotone', 'salernitana', 'empoli', 'venezia', 'como', 'parma', 'frosinone', 'atalanta', 'sampdoria', 'inter', 'sassuolo', 'brescia', 'lecce', 'cagliari', 'napoli', 'chievo', 'monza'}
31
True


In [ ]:
# Now, let us work on the date and time fields

# Let us ensure no null date/time entries

null_count = league_matches['date'].isna().sum() + league_matches['time'].isna().sum()

print(null_count)

# The date format is fine so let us convert the time instead

# Let us get all time's to see if its a 24hr or 12hr format

print(league_matches['time'].unique())


# Let us confirm there are no null values
null_count = league_matches['date'].isna().sum() + league_matches['time'].isna().sum()

print(null_count)

0
['14:45' '12:00' '09:00' '06:30' '12:30' '15:45' '13:00' '10:00' '13:30'
 '07:30' '14:00' '15:00' '14:30' '10:30' '08:30' '11:00' '11:15' '12:45'
 '14:15' '16:00' '13:45' '14:50' '12:05' '09:05' '06:35' '12:15' '12:55'
 '15:30']
0


In [37]:
# Let us confirm the data types

league_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2788 entries, 0 to 2787
Data columns (total 62 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   comp                                2788 non-null   object        
 1   round                               2788 non-null   int64         
 2   home_team                           2788 non-null   object        
 3   away_team                           2788 non-null   object        
 4   home_goals                          2788 non-null   int64         
 5   away_goals                          2788 non-null   int64         
 6   date                                2788 non-null   datetime64[ns]
 7   time                                2788 non-null   object        
 8   home_expected_goals_(xg)            755 non-null    float64       
 9   away_expected_goals_(xg)            755 non-null    float64       
 10  home_ball_possession    

In [38]:
# next, let us handle the expected goal columns for home and away. They should be nullable float ('Float64') types

# null count
print(f"Total null values in xG columns: {league_matches['home_expected_goals_(xg)'].isna().sum() + league_matches['away_expected_goals_(xg)'].isna().sum()}")

# rename and convert
league_matches['home_xg'] = pd.to_numeric(league_matches['home_expected_goals_(xg)'], errors='coerce').astype('Float64')
league_matches['away_xg'] = pd.to_numeric(league_matches['away_expected_goals_(xg)'], errors='coerce').astype('Float64')

# sanity check
print(f"Total null values in xG columns: {league_matches['home_xg'].isna().sum() + league_matches['away_xg'].isna().sum()}")

# drop original columns
league_matches = league_matches.drop(['home_expected_goals_(xg)', 'away_expected_goals_(xg)'], axis=1)


# next, let us handle the ball possesion fields. They should be non-nullable float types('float64') types

# null count
print(f"Total null values in ball possession columns: {league_matches['away_ball_possession'].isna().sum() + league_matches['away_ball_possession'].isna().sum()}")

# rename and convert
league_matches['away_ball_possession'] = league_matches['away_ball_possession'].astype('str').str.replace("%", "").astype("int")/100
league_matches['home_ball_possession'] = league_matches['home_ball_possession'].astype('str').str.replace("%", "").astype("int")/100

# sanity check
print(f"Total null values in ball possession columns: {league_matches['away_ball_possession'].isna().sum() + league_matches['away_ball_possession'].isna().sum()}")

# the following columns should have wrong datatypes (float=>Int8): home_blocked_shots, away_blocked_shots, home_big_chances, 'home_headed_goals', 'away_headed_goals'
# away_big_chances, 'home_shots_inside_the_box', 'away_shots_inside_the_box', 'home_shots_outside_the_box', 'away_shots_outside_the_box', 'home_hit_the_woodwork', 'away_hit_the_woodwork', 
league_matches[['home_blocked_shots', 'away_blocked_shots', 'home_big_chances', 'away_big_chances', 'home_shots_inside_the_box', 'away_shots_inside_the_box', 'home_shots_outside_the_box', 
'away_shots_outside_the_box', 'home_hit_the_woodwork', 'away_hit_the_woodwork', 'home_headed_goals', 'away_headed_goals', 'home_free_kicks', 'away_free_kicks', 'home_yellow_cards', 'away_yellow_cards', 
'home_throw-ins', 'away_throw-ins', 'home_touches_in_the_opposition_box', 'away_touches_in_the_opposition_box', 'home_clearances_total', 'away_clearances_total', 'home_interceptions', 
'away_interceptions', 'home_red_cards', 'away_red_cards']] = league_matches[['home_blocked_shots', 'away_blocked_shots', 'home_big_chances', 'away_big_chances', 'home_shots_inside_the_box', 'away_shots_inside_the_box', 'home_shots_outside_the_box', 
'away_shots_outside_the_box', 'home_hit_the_woodwork', 'away_hit_the_woodwork', 'home_headed_goals', 'away_headed_goals', 'home_free_kicks', 'away_free_kicks', 'home_yellow_cards', 'away_yellow_cards', 
'home_throw-ins', 'away_throw-ins', 'home_touches_in_the_opposition_box', 'away_touches_in_the_opposition_box', 'home_clearances_total', 'away_clearances_total', 'home_interceptions', 
'away_interceptions', 'home_red_cards', 'away_red_cards']].astype("Int8")

# process special columns (string=>float/int): 'home_passes', 'away_passes', 'home_passes_in_the_final_third', 'away_passes_in_the_final_third', 'home_crosses', 'away_crosses', 'home_tackles', 'away_tackles'

#sanity check: what do they look like? 
league_matches[['home_passes', 'away_passes', 'home_passes_in_the_final_third', 'away_passes_in_the_final_third', 'home_crosses', 'away_crosses', 'home_tackles', 'away_tackles']].head()


# next, let us handle the passes fields. The original passes column contains three metrics - pass_accuracy, passes_completed, passes_attempted - (str => Float64, Int64, Int64)

# null count 
print(f"Total null values in passes columns : {league_matches['home_passes'].isna().sum() + league_matches['away_passes'].isna().sum()}")

# extract and convert pass accuracy
league_matches['home_pass_accuracy'] = league_matches['home_passes'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['home_pass_accuracy'] = pd.to_numeric(league_matches['home_pass_accuracy'], errors='coerce').astype("Int64")/100
league_matches['away_pass_accuracy'] = league_matches['away_passes'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['away_pass_accuracy'] = pd.to_numeric(league_matches['away_pass_accuracy'], errors='coerce').astype('Int64')/100

# extract and convert passes completed and passes attempted
league_matches['away_passes_completed'] = league_matches['away_passes'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_passes_completed'] = league_matches['home_passes'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_passes_completed'] = pd.to_numeric(league_matches['home_passes_completed'], errors='coerce').astype('Int64')
league_matches['away_passes_completed'] = pd.to_numeric(league_matches['away_passes_completed'], errors='coerce').astype('Int64')
league_matches['home_passes_attempted'] = league_matches['home_passes'].str.extract(r'\(\d+/(\d+)\)')
league_matches['away_passes_attempted'] = league_matches['away_passes'].str.extract(r'\(\d+/(\d+)\)')
league_matches['home_passes_attempted'] = pd.to_numeric(league_matches['home_passes_attempted'], errors='coerce').astype('Int64')
league_matches['away_passes_attempted'] = pd.to_numeric(league_matches['away_passes_attempted'], errors='coerce').astype('Int64')

# sanity check: do I still have the same null-count? 
print(f"Total null values in pass accuracy columns : {league_matches['home_pass_accuracy'].isna().sum() + league_matches['away_pass_accuracy'].isna().sum()}")
print(f"Total null values in pass completed columns : {league_matches['home_passes_completed'].isna().sum() + league_matches['away_passes_completed'].isna().sum()}")
print(f"Total null values in pass attempted columns : {league_matches['home_passes_attempted'].isna().sum() + league_matches['away_passes_attempted'].isna().sum()}")


# next, let us handle the passes in final third fields. The original passes in final third column contains three metrics - passes_accuracy, passes_completed, passes_attempted - (str => Float64, Int64, Int64)

# null count
print(f"Total null values in passes in final third column: {league_matches['home_passes_in_the_final_third'].isna().sum() + league_matches['away_passes_in_the_final_third'].isna().sum()}")

# extract and convert pass accuracy
league_matches['home_final_third_pass_accuracy'] = league_matches['home_passes_in_the_final_third'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['home_final_third_pass_accuracy'] = pd.to_numeric(league_matches['home_final_third_pass_accuracy'], errors='coerce').astype("Int64")/100
league_matches['away_final_third_pass_accuracy'] = league_matches['away_passes_in_the_final_third'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['away_final_third_pass_accuracy'] = pd.to_numeric(league_matches['away_final_third_pass_accuracy'], errors='coerce').astype('Int64')/100

# extract and convert passes attempted and completed
league_matches['home_final_third_passes_completed'] = league_matches['home_passes_in_the_final_third'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_final_third_passes_attempted'] = league_matches['home_passes_in_the_final_third'].str.extract(r'\(\d+/(\d+)\)')
league_matches['away_final_third_passes_completed'] = league_matches['away_passes_in_the_final_third'].str.extract(r'\((\d+)/\d+\)')
league_matches['away_final_third_passes_attempted'] = league_matches['away_passes_in_the_final_third'].str.extract(r'\(\d+/(\d+)\)')
league_matches['home_final_third_passes_completed'] = pd.to_numeric(league_matches['home_final_third_passes_completed'], errors='coerce').astype('Int64')
league_matches['home_final_third_passes_attempted'] = pd.to_numeric(league_matches['home_final_third_passes_attempted'], errors='coerce').astype('Int64')
league_matches['away_final_third_passes_completed'] = pd.to_numeric(league_matches['away_final_third_passes_completed'], errors='coerce').astype('Int64')
league_matches['away_final_third_passes_attempted'] = pd.to_numeric(league_matches['away_final_third_passes_attempted'], errors='coerce').astype('Int64')

# sanity check: do I still have the same null-count?
print(f"Total null values in passes in final third accuracy columns : {league_matches['home_final_third_pass_accuracy'].isna().sum() + league_matches['away_final_third_pass_accuracy'].isna().sum()}")
print(f"Total null values in passes in final third completed columns : {league_matches['home_final_third_passes_completed'].isna().sum() + league_matches['away_final_third_passes_completed'].isna().sum()}")
print(f"Total null values in passes in final third attempted columns : {league_matches['home_final_third_passes_attempted'].isna().sum() + league_matches['away_final_third_passes_attempted'].isna().sum()}")

# next, let us handle the crosses fields. The original crosses column contains three metrics - crosses_accuracy, crosses_completed, crosses_attempted - (str => Float64, Int64, Int64)

# null count 
print(f"Total null values in crosses columns : {league_matches['home_crosses'].isna().sum() + league_matches['away_crosses'].isna().sum()}")

# extract and convert pass accuracy
league_matches['home_cross_accuracy'] = league_matches['home_crosses'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['home_cross_accuracy'] = pd.to_numeric(league_matches['home_cross_accuracy'], errors='coerce').astype("Int64")/100
league_matches['away_cross_accuracy'] = league_matches['away_crosses'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['away_cross_accuracy'] = pd.to_numeric(league_matches['away_cross_accuracy'], errors='coerce').astype('Int64')/100

# extract and convert passes completed and passes attempted
league_matches['away_crosses_completed'] = league_matches['away_crosses'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_crosses_completed'] = league_matches['home_crosses'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_crosses_completed'] = pd.to_numeric(league_matches['home_crosses_completed'], errors='coerce').astype('Int64')
league_matches['away_crosses_completed'] = pd.to_numeric(league_matches['away_crosses_completed'], errors='coerce').astype('Int64')
league_matches['home_crosses_attempted'] = league_matches['home_crosses'].str.extract(r'\(\d+/(\d+)\)')
league_matches['away_crosses_attempted'] = league_matches['away_crosses'].str.extract(r'\(\d+/(\d+)\)')
league_matches['home_crosses_attempted'] = pd.to_numeric(league_matches['home_crosses_attempted'], errors='coerce').astype('Int64')
league_matches['away_crosses_attempted'] = pd.to_numeric(league_matches['away_crosses_attempted'], errors='coerce').astype('Int64')

# sanity check: do I still have the same null-count? 
print(f"Total null values in cross accuracy columns : {league_matches['home_cross_accuracy'].isna().sum() + league_matches['away_cross_accuracy'].isna().sum()}")
print(f"Total null values in cross completed columns : {league_matches['home_crosses_completed'].isna().sum() + league_matches['away_crosses_completed'].isna().sum()}")
print(f"Total null values in cross attempted columns : {league_matches['home_crosses_attempted'].isna().sum() + league_matches['away_crosses_attempted'].isna().sum()}")

# next, let us handle the tackles fields. The original tackles column contains three metrics - tackle_success_rate, tackles_won, tackles_accepted

# null count
print(f"Total null values in tackles columns : {league_matches['home_tackles'].isna().sum() + league_matches['away_tackles'].isna().sum()}")

# extract and convert tackle success rates
league_matches['home_tackle_success_rate'] = league_matches['home_tackles'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['home_tackle_success_rate'] = pd.to_numeric(league_matches['home_tackle_success_rate'], errors='coerce').astype("Int64")/100
league_matches['away_tackle_success_rate'] = league_matches['away_tackles'].astype('str').str.split(" ").str[0].str.replace("%", "")
league_matches['away_tackle_success_rate'] = pd.to_numeric(league_matches['away_tackle_success_rate'], errors='coerce').astype('Int64')/100

# extract and convert tackles won and tackles attempted
league_matches['home_tackles_won'] = league_matches['home_tackles'].str.extract(r'\((\d+)/\d+\)')
league_matches['home_tackles_attempted'] = league_matches['home_tackles'].str.extract(r'\(\d+/(\d+)\)')
league_matches['away_tackles_won'] = league_matches['away_tackles'].str.extract(r'\((\d+)/\d+\)')
league_matches['away_tackles_attempted'] = league_matches['away_tackles'].str.extract(r'\(\d+/(\d+)\)')
league_matches['home_tackles_won'] = pd.to_numeric(league_matches['home_tackles_won'], errors='coerce').astype('Int64')
league_matches['home_tackles_attempted'] = pd.to_numeric(league_matches['home_tackles_attempted'], errors='coerce').astype('Int64')
league_matches['away_tackles_won'] = pd.to_numeric(league_matches['away_tackles_won'], errors='coerce').astype('Int64')
league_matches['away_tackles_attempted'] = pd.to_numeric(league_matches['away_tackles_attempted'], errors='coerce').astype('Int64')

# sanity check: do I still have the same null-count? 
print(f"Total null values in tackle success rate columns : {league_matches['home_tackle_success_rate'].isna().sum() + league_matches['away_tackle_success_rate'].isna().sum()}")
print(f"Total null values in tackles won columns : {league_matches['home_tackles_won'].isna().sum() + league_matches['away_tackles_won'].isna().sum()}")
print(f"Total null values in tackles attempted columns : {league_matches['home_tackles_attempted'].isna().sum() + league_matches['away_tackles_attempted'].isna().sum()}")


# drop the old columns
league_matches.drop(['home_passes', 'away_passes', 'home_passes_in_the_final_third', 'away_passes_in_the_final_third', 'home_crosses', 'away_crosses', 'home_tackles', 'away_tackles'], axis=1)

# confirm the current structure of the dataframe
league_matches.info()

Total null values in xG columns: 4066
Total null values in xG columns: 4066
Total null values in ball possession columns: 0
Total null values in ball possession columns: 0
Total null values in passes columns : 3602
Total null values in pass accuracy columns : 3602
Total null values in pass completed columns : 3602
Total null values in pass attempted columns : 3602
Total null values in passes in final third column: 5396
Total null values in passes in final third accuracy columns : 5396
Total null values in passes in final third completed columns : 5396
Total null values in passes in final third attempted columns : 5396
Total null values in crosses columns : 5396
Total null values in cross accuracy columns : 5396
Total null values in cross completed columns : 5396
Total null values in cross attempted columns : 5396
Total null values in tackles columns : 5396
Total null values in tackle success rate columns : 5396
Total null values in tackles won columns : 5396
Total null values in tackle

In [39]:
# now, let us start extracting the erd's 

import json
import os 

league_file_path = "../data/leagues.json"

if os.path.exists(league_file_path):
    print("league file exists")
else:
    with open(league_file_path, 'w') as f:
        json.dump({}, f)

with open(league_file_path, 'r') as f:
    leagues_json = json.load(f)

curr_league = league_matches.iloc[0]['comp'].lower().replace(" ", "_")

print(curr_league)

if curr_league in leagues_json:
    pass
else:
    if not leagues_json:
        leagues_json[curr_league] = {"id": 0, "name": curr_league, "country": league_countries[curr_league]}
    else:
        max_id = -float('inf')
        for league in leagues_json.values():
            if league['id'] > max_id:
                max_id = league['id']
        leagues_json[curr_league] = {"id": max_id+1, "name": curr_league, "country": league_countries[curr_league]}
print(leagues_json)

with open(league_file_path, 'w') as f:
    json.dump(leagues_json, f)

league file exists
serie_a
{'bundesliga': {'id': 0, 'name': 'bundesliga', 'country': 'Germany'}, 'premier_league': {'id': 1, 'name': 'premier_league', 'country': 'England'}, 'serie_a': {'id': 2, 'name': 'serie_a', 'country': 'Italy'}}


In [40]:
# next, we want to generate the seasons
import json
import os 

print(curr_league)

league_dir = f"../data/leagues/{curr_league}"
os.makedirs(league_dir, exist_ok=True)

season_file_path = f"{league_dir}/seasons.json"
if not os.path.exists(season_file_path):
    with open(season_file_path, 'w') as f:
        json.dump({},f)

with open(season_file_path, 'r') as f:
    seasons_object = json.load(f)

all_seasons = sorted(league_matches['season'].unique())

with open("../data/leagues.json", 'r') as f:
    league_id = json.load(f)[curr_league]['id']

next_season_id = 0
if seasons_object:
    next_id = max(season_data['id'] for season_data in seasons_object.values()) + 1

for season in all_seasons:
    if season in seasons_object:
        continue
    else:
        seasons_object[season] = {
            "id": next_season_id,
            "name": season,
            "league_id": league_id,
            "status": "completed" if season != "2024_2025" else "active"
        }
        next_season_id += 1

with open(season_file_path, 'w') as f:
    json.dump(seasons_object, f, indent=4)

serie_a


In [41]:
# Custom JSON encoder to handle non-serializable types
class PandasNumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        # Handle NumPy types
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        # Handle pandas Timestamp
        elif isinstance(obj, pd.Timestamp):
            return obj.strftime('%Y-%m-%d')
        # Handle pandas extension types
        elif pd.api.types.is_integer_dtype(type(obj)):
            return int(obj) if pd.notna(obj) else None
        elif pd.api.types.is_float_dtype(type(obj)):
            return float(obj) if pd.notna(obj) else None
        return super().default(obj)
        # The line return super().default(obj) in your custom encoder means "fall back to the parent class's implementation of the default() method if none of my custom handling applies."


In [42]:
# now, we want to generate the match data
import json
import os

print(curr_league)

league_dir = f"../data/leagues/{curr_league}"
os.makedirs(league_dir, exist_ok=True)

# get league_id and season_id
with open(league_dir + "/seasons.json", 'r') as f:
    seasons_object = json.load(f)
with open("../data/leagues.json", 'r') as f:
    league_id = json.load(f)[curr_league]['id']
seasons_id_map = {season_name: season_data['id'] for season_name, season_data in seasons_object.items()}

for (season, round_num), group in league_matches.groupby(['season', 'round']):
    season_id = seasons_id_map[season]
    matches = group[['date', 'time', 'away_team', 'home_team', 'home_goals', 'away_goals']].to_dict('records')

    for i, match in enumerate(matches):
        if isinstance(match['date'], pd.Timestamp):
            match['date'] = match['date'].strftime('%Y-%m-%d')

        match['id'] = i
        match['season_id'] = season_id
        match['league_id'] = league_id
        match['round'] = round_num
        if match['home_goals'] > match['away_goals']:
            match['result'] = 1
        elif match['home_goals'] == match['away_goals']:
            match['result'] = 0
        else:
            match['result'] = 2
        match['home_team_name'] = team_dict[match.pop('home_team')]
        match['away_team_name'] = team_dict[match.pop('away_team')]
        match['home_team_id'] = None
        match['away_team_id'] = None

    season_key = season.lower().replace(" ", "_")
    round_dir = f"../data/leagues/{curr_league}/{season_key}/rounds"
    os.makedirs(round_dir, exist_ok=True)

    with open(f"{round_dir}/round_{round_num}.json", 'w') as f:
        json.dump(matches, f, indent=4, cls=PandasNumpyEncoder)

serie_a


In [43]:
# next, lets generate a league_info.json for serie a
league_dir = f"../data/leagues/{curr_league}"
os.makedirs(league_dir, exist_ok=True)

league_info = {
    "id": league_id,
    "name": curr_league.replace("_", " ").title(),  # Convert from slug to proper name
    "country": league_countries[curr_league],
    "founded": None,  # Add if you have this data
    "num_teams": len(team_dict),  # Count of unique teams
    "team_name_mappings": team_dict,  # Include your name mappings
}

# Write to file
info_path = f"{league_dir}/info.json"
with open(info_path, 'w') as f:
    json.dump(league_info, f, indent=4, cls=PandasNumpyEncoder)

In [44]:
# Process each season
for season in seasons_object:
    season_key = season.lower().replace(" ", "_").replace("/", "_")
    season_path = f"{league_dir}/{season_key}"
    os.makedirs(season_path, exist_ok=True)
    
    # Find all teams that played in this season
    season_matches = league_matches[league_matches['season'] == season]
    season_teams = set(season_matches['home_team'].tolist() + season_matches['away_team'].tolist())
    
    # Create teams.json for this season
    teams_data = {}
    team_id = 0
    for team in season_teams:
        team_key = team.lower().replace(" ", "_").replace(".", "")
        teams_data[team_key] = {
            "id": team_id,
            "name": team,  # Store full proper name
            "country": league_countries[curr_league],  # Default to league country
            "participated": True
        }
        team_id += 1
    
    # Write teams.json
    with open(f"{season_path}/teams.json", 'w') as f:
        json.dump(teams_data, f, indent=4, cls=PandasNumpyEncoder)

In [45]:
# For each season, create season_info.json
for season_name, season_data in seasons_object.items():
    season_key = season_name.lower().replace(" ", "_").replace("/", "_")
    season_path = f"{league_dir}/{season_key}"
    
    # Calculate season stats from matches
    season_matches = league_matches[league_matches['season'] == season_name]
    
    # Create season info
    season_info = {
        "id": season_data["id"],
        "name": season_name,
        "league_id": season_data["league_id"],
        "start_date": season_matches['date'].min().strftime('%Y-%m-%d'),
        "end_date": season_matches['date'].max().strftime('%Y-%m-%d'),
        "status": season_data["status"],
        "num_teams": len(set(season_matches['home_team'].tolist() + season_matches['away_team'].tolist())),
        "total_matches": len(season_matches),
        "rounds": season_matches['round'].max()
    }
    
    # Write season_info.json
    with open(f"{season_path}/season_info.json", 'w') as f:
        json.dump(season_info, f, indent=4, cls=PandasNumpyEncoder)

In [46]:
# For simplicity, let us assign ids to every team that exists in a league and add it to the info.json file which holds an overview of league data

with open(f'../data/leagues/{curr_league}/info.json', 'r') as f:
    league_object = json.load(f)
print(league_object['team_name_mappings'].keys())

team_names = sorted(league_object['team_name_mappings'].keys())

league_object['teams'] = {}

for i, team_key in enumerate(team_names):
    league_object['teams'][team_key] = {
        'id': i,
        'full_name': league_object['team_name_mappings'][team_key],
        'short_name': team_key
    }

if 'team_name_mappings' in league_object:
    del league_object['team_name_mappings']

with open(f'../data/leagues/{curr_league}/info.json', 'w') as f:
    json.dump(league_object, f, indent=4)

dict_keys(['ac_milan', 'as_roma', 'atalanta', 'benevento', 'bologna', 'brescia', 'cagliari', 'chievo', 'como', 'cremonese', 'crotone', 'empoli', 'fiorentina', 'frosinone', 'genoa', 'inter', 'juventus', 'lazio', 'lecce', 'monza', 'napoli', 'parma', 'salernitana', 'sampdoria', 'sassuolo', 'spal', 'spezia', 'torino', 'udinese', 'venezia', 'verona'])


In [47]:
# Now, let's update the team_id across our data entries

# We need to update the teams.json files across each season, as well as the team_ids inside the round_<n>.json files inside each <season>/rounds/

base_dir = f"../data/leagues/{curr_league}"

# First, lets get the id mappings from info.json

with open(base_dir+"/info.json", 'r') as f:
    league_info = json.load(f)

# get the ids
team_id_mappings = {team_key: team_data['id'] for team_key, team_data in league_info['teams'].items()}

# To replace all teams.json inside all the seasons we need to first find all seasons in our base directory

seasons = [season for season in os.listdir(base_dir) if os.path.isdir(f'{base_dir}/{season}') and season != "rounds"]

print(seasons)

# Now, we've gotten every season. We want to update the teams.json - teams.json are directly in the seasons directory

for season in seasons:
    season_dir = f'{base_dir}/{season}'

    teams_file = f'{season_dir}/teams.json'
    if os.path.exists(teams_file):
        with open(teams_file, 'r') as f:
            teams_data = json.load(f)
        
        # Update team IDs in the teams.json file
        updated_teams = {}
        for team_key, team_data in teams_data.items():
            if team_key in team_id_mappings:
                team_data['id'] = team_id_mappings[team_key]
            updated_teams[team_key] = team_data
        # Save the updated teams data
        with open(teams_file, 'w') as f:
            json.dump(updated_teams, f, indent=4)
    
    # Now, let us update all round files
    rounds_dir = f'{season_dir}/rounds'

    if os.path.exists(rounds_dir):
        round_files = [f for f in os.listdir(rounds_dir) if f.startswith("round_") and f.endswith(".json")]

        for round_file in round_files:
            round_path = f'{rounds_dir}/{round_file}'

            with open(round_path, 'r') as f:
                matches = json.load(f)
            
            for m in matches:
                # Get team names from the match data

                home_team_name = m.get("home_team_name")
                away_team_name = m.get("away_team_name")

                # Find the team keys from full names
                home_team_key = None
                away_team_key = None

                for team_key, team_data in league_info['teams'].items():
                    if team_data['full_name'] == home_team_name:
                        home_team_key = team_key
                    if team_data['full_name'] == away_team_name:
                        away_team_key = team_key
                
                # Update team IDs if keys were found
                if home_team_key and home_team_key in team_id_mappings:
                    m["home_team_id"] = team_id_mappings[home_team_key]
                if away_team_key and away_team_key in team_id_mappings:
                    m["away_team_id"] = team_id_mappings[away_team_key]
            
            # Save the updated matches
            with open(round_path, 'w') as f:
                json.dump(matches, f, indent=4)

['2021_2022', '2019_2020', '2018_2019', '2017_2018', '2020_2021', '2024_2025', '2023_2024', '2022_2023']


In [48]:
import pandas as pd
import json
import os
from datetime import datetime

# Base directory and current league
base_dir = "../data/leagues/serie_a"
curr_league = "serie_a"

# Load league info for team mappings
with open(f"{base_dir}/info.json", 'r') as f:
    league_info = json.load(f)
team_mappings = league_info.get("teams", {})

# Ensure seasons

all_league_seasons = {}
with open(f"{base_dir}/seasons.json", 'r') as f:
    all_league_seasons = json.load(f)

# ['2021_2022', '2019_2020', '2015_2016', '2018_2019', '2017_2018', '2020_2021', '2016_2017', '2024_2025', '2023_2024', '2022_2023']
print(seasons)
print(all_league_seasons)

# Initialize dictionary to collect team stats by season
team_stats_by_season = {}

# Global counter for match IDs (only for relating home and away stats)
match_id_counter = 0

# Process each match and split into team-specific records
print("Processing matches using itertuples()...")

for match in league_matches.itertuples():
    # Increment match ID for this match
    match_id_counter += 1
    
    # Extract season
    season = match.season
    
    # Initialize season dict if needed
    if season not in team_stats_by_season:
        team_stats_by_season[season] = {}
    
    # Get team info
    home_team_key = match.home_team
    away_team_key = match.away_team
    
    # Get team IDs
    home_team_id = team_mappings.get(home_team_key, {}).get('id')
    away_team_id = team_mappings.get(away_team_key, {}).get('id')
    
    # Get team names for readability
    home_team_name = team_mappings.get(home_team_key, {}).get('full_name', home_team_key)
    away_team_name = team_mappings.get(away_team_key, {}).get('full_name', away_team_key)
    
    # Determine match result and points
    if match.home_goals > match.away_goals:
        home_result, away_result = "win", "loss"
        home_points, away_points = 3, 0
    elif match.home_goals < match.away_goals:
        home_result, away_result = "loss", "win"
        home_points, away_points = 0, 3
    else:
        home_result = away_result = "draw"
        home_points = away_points = 1
    
    # Format date if needed
    match_date = match.date
    if isinstance(match_date, pd.Timestamp):
        match_date = match_date.strftime('%Y-%m-%d')
    
    # Process both teams in one go
    for is_home, team_key, team_id, team_name, opponent_key, opponent_id, opponent_name, \
        result, points, goals_scored, goals_conceded in [
        # Home team data
        (True, home_team_key, home_team_id, home_team_name, away_team_key, away_team_id, away_team_name,
         home_result, home_points, match.home_goals, match.away_goals),
        # Away team data
        (False, away_team_key, away_team_id, away_team_name, home_team_key, home_team_id, home_team_name,
         away_result, away_points, match.away_goals, match.home_goals)
    ]:
        # Create the team stats record
        team_stats = {
            # Match reference
            "match_id": match_id_counter,
            "season_id": all_league_seasons[season]['id'],
            "season": season,
            "round": match.round,
            "date": match_date,
            
            # Team information
            "team_id": team_id,
            "team_key": team_key,
            "team_name": team_name,
            "is_home": is_home,
            
            # Opponent information
            "opponent_id": opponent_id,
            "opponent_key": opponent_key,
            "opponent_name": opponent_name,
            
            # Match outcome
            "result": result,
            "points": points,
            "goals_scored": goals_scored,
            "goals_conceded": goals_conceded
        }
        
        # Add stats specific to home or away team
        prefix = "home_" if is_home else "away_"
        
        # Core stats that are always present
        for base_stat in ["ball_possession", "goal_attempts", "shots_on_goal", 
                          "shots_off_goal", "corner_kicks", "offsides", 
                          "fouls", "goalkeeper_saves"]:
            # Get the column with appropriate prefix
            col_name = f"{prefix}{base_stat}"
            if hasattr(match, col_name):
                # Remove prefix and standardize name
                stat_name = base_stat.replace("_", "_") 
                team_stats[stat_name] = getattr(match, col_name)
        
        # Handle expected goals (xG) if present
        xg_col = f"{prefix}xg"
        value = getattr(match, xg_col, None)
        team_stats["xg"] = None if pd.isna(value) else value
        
        # Optional stats that might be null
        for opt_stat in ["blocked_shots", "big_chances", "shots_inside_the_box", 
                        "shots_outside_the_box", "hit_the_woodwork", "headed_goals",
                        "free_kicks", "yellow_cards", "red_cards", "throw-ins",
                        "touches_in_the_opposition_box", "clearances_total", 
                        "interceptions"]:
            col_name = f"{prefix}{opt_stat}"
            stat_name = opt_stat.replace("the_", "").replace("_total", "")
            team_stats[stat_name] = None
            
            if hasattr(match, col_name):
                # Standardize name (remove prefix, clean up)
                stat_name = opt_stat.replace("the_", "")
                team_stats[stat_name] = None if pd.isna(getattr(match, col_name)) else getattr(match, col_name)
        
        # Passing stats
        pass_cols = {
            "passes_completed": f"{prefix}passes_completed",
            "passes_attempted": f"{prefix}passes_attempted",
            "pass_accuracy": f"{prefix}pass_accuracy"
        }
        
        # Always create these fields, regardless of whether the match has these attributes
        for stat_name, col_name in pass_cols.items():
            # Initialize with None by default
            value = getattr(match, col_name, None)

            # Assign if not null
            team_stats[stat_name] = value if pd.notna(value) else None

        # Final third passing stats
        third_pass_cols = {
            "final_third_passes_completed": f"{prefix}final_third_passes_completed",
            "final_third_passes_attempted": f"{prefix}final_third_passes_attempted",
            "final_third_pass_accuracy": f"{prefix}final_third_pass_accuracy"
        }

        # Always create these fields, regardless of whether the match has these attributes
        for stat_name, col_name in third_pass_cols.items():
            value = getattr(match, col_name, None)
            team_stats[stat_name] = value if pd.notna(value) else None

        
        
        # Cross stats
        cross_cols = {
            "crosses_completed": f"{prefix}crosses_completed",
            "crosses_attempted": f"{prefix}crosses_attempted",
            "cross_accuracy": f"{prefix}cross_accuracy"
        }
        for stat_name, col_name in cross_cols.items():
            value = getattr(match, col_name, None)
            team_stats[stat_name] = value if pd.notna(value) else None
        
        # Tackle stats
        tackle_cols = {
            "tackles_won": f"{prefix}tackles_won",
            "tackles_attempted": f"{prefix}tackles_attempted",
            "tackle_success_rate": f"{prefix}tackle_success_rate"
        }
        for stat_name, col_name in tackle_cols.items():
            value = getattr(match, col_name, None)
            team_stats[stat_name] = value if pd.notna(value) else None
        
        # Add derived statistics
        
        # Shot efficiency
        if team_stats.get("goal_attempts", 0) > 0:
            team_stats["shot_accuracy"] = team_stats.get("shots_on_goal", 0) / team_stats["goal_attempts"]
            team_stats["goal_efficiency"] = goals_scored / team_stats["goal_attempts"]

        else:
            team_stats["shot_accuracy"] = 0
            team_stats["goal_efficiency"] = 0
        
        # Conversion rate (goals per shot on target)
        if team_stats.get("shots_on_goal", 0) > 0:
            team_stats["conversion_rate"] = goals_scored / team_stats["shots_on_goal"]
        else:
            team_stats["conversion_rate"] = 0
        
        # xG efficiency
        if "xg" in team_stats and pd.notna(team_stats['xg']) and team_stats["xg"] > 0:
            team_stats["xg_difference"] = goals_scored - team_stats["xg"]
            team_stats["xg_efficiency"] = goals_scored / team_stats["xg"]
        else:
            team_stats["xg_difference"] = None
            team_stats["xg_efficiency"] = None
        

        
        # Initialize team's stats list if it doesn't exist
        if team_key not in team_stats_by_season[season]:
            team_stats_by_season[season][team_key] = []
        
        # Add to team's stats
        team_stats_by_season[season][team_key].append(team_stats)

# Now team_stats_by_season contains all the transformed data
print(f"Processed {match_id_counter} matches")

# Save team-specific data to files
print("Saving team statistics to files...")
for season, teams in team_stats_by_season.items():
    for team_key, matches in teams.items():
        # Create team directory if it doesn't exist
        team_dir = f"{base_dir}/{season}/teams/{team_key}"
        os.makedirs(team_dir, exist_ok=True)
        
        # Sort matches by round
        sorted_matches = sorted(matches, key=lambda m: pd.to_datetime(m["round"]))
        
        # Save match_stats.json
        with open(f"{team_dir}/match_stats.json", 'w') as f:
            json.dump(sorted_matches, f, indent=4, cls=PandasNumpyEncoder)
        
        # Calculate and save season totals
        # [Code for calculating season aggregates would go here]
        
        print(f"Saved team stats for {team_key} in {season}")

print("Team statistics processing completed!")

['2021_2022', '2019_2020', '2018_2019', '2017_2018', '2020_2021', '2024_2025', '2023_2024', '2022_2023']
{'2017_2018': {'id': 0, 'name': '2017_2018', 'league_id': 2, 'status': 'completed'}, '2018_2019': {'id': 1, 'name': '2018_2019', 'league_id': 2, 'status': 'completed'}, '2019_2020': {'id': 2, 'name': '2019_2020', 'league_id': 2, 'status': 'completed'}, '2020_2021': {'id': 3, 'name': '2020_2021', 'league_id': 2, 'status': 'completed'}, '2021_2022': {'id': 4, 'name': '2021_2022', 'league_id': 2, 'status': 'completed'}, '2022_2023': {'id': 5, 'name': '2022_2023', 'league_id': 2, 'status': 'completed'}, '2023_2024': {'id': 6, 'name': '2023_2024', 'league_id': 2, 'status': 'completed'}, '2024_2025': {'id': 7, 'name': '2024_2025', 'league_id': 2, 'status': 'active'}}
Processing matches using itertuples()...
Processed 2788 matches
Saving team statistics to files...
Saved team stats for como in 2024_2025
Saved team stats for udinese in 2024_2025
Saved team stats for inter in 2024_2025
Saved

In [50]:
# here, lets include the derived statistics & team_form calculations 
# here, we only have the function definitions
import os
import json
import pandas as pd
import shutil

def safe_divide(numerator, denominator):
    """Safely divide two numbers, handling division by zero"""
    return numerator / denominator if denominator != 0 else 0

def calculate_team_form(matches, current_round=None, scale_min=1, scale_max=5):
    """
    Calculate a team's form on a 1-5 scale based on recent matches.
    
    Parameters:
    matches (list): List of match dictionaries with team statistics
    current_round (int): The round number to calculate form up to (excluding this round)
    scale_min (float): Minimum value of the form scale
    scale_max (float): Maximum value of the form scale
    
    Returns:
    float: Team form rating on the specified scale
    dict: Component breakdowns of the form calculation
    """
    # Sort matches by round in descending order (most recent first)
    sorted_matches = sorted(matches, key=lambda x: x['round'], reverse=True)
    
    # Filter matches up to the current round if specified
    if current_round is not None:
        sorted_matches = [m for m in sorted_matches if m['round'] < current_round]
    
    # Take up to 5 most recent matches
    recent_matches = sorted_matches[:5]
    
    # Handle early season with fewer matches
    if len(recent_matches) == 0:
        return (scale_min + scale_max) / 2, {"note": "No matches played yet"}
    
    # Define weights - most recent has highest weight
    weights = [5, 4, 3, 2, 1][:len(recent_matches)]
    total_weight = sum(weights)
    
    # Initialize component dictionaries
    components = {
        "matches_used": len(recent_matches),
        "match_details": [],
        "results_component": 0,
        "goals_component": 0,
        "shots_component": 0,
        "defense_component": 0,
        "possession_component": 0
    }
    
    # Calculate form components
    for i, match in enumerate(recent_matches):
        weight = weights[i]
        
        # 1. Results Component (W=3, D=1, L=0)
        match_points = 3 if match['result'] == 'win' else (1 if match['result'] == 'draw' else 0)
        results_contribution = (match_points * weight) / total_weight
        
        # 2. Goals Component - based on goal difference
        goal_diff = match['goals_scored'] - match['goals_conceded']
        # Scale: +/-3 goals = max impact
        goal_diff_scaled = max(min(goal_diff, 3), -3) / 3  # Range from -1 to 1
        goals_contribution = (goal_diff_scaled * weight) / total_weight
        
        # 3. Shots Component - based on shot accuracy and conversion
        if match.get('shots_on_goal', 0) > 0 and match.get('goal_attempts', 0) > 0:
            shot_accuracy = match['shots_on_goal'] / match['goal_attempts']
            conversion = match['goals_scored'] / match['shots_on_goal'] if match['shots_on_goal'] > 0 else 0
            shots_quality = (shot_accuracy + min(conversion, 1)) / 2  # Range 0 to 1
        else:
            shots_quality = 0
        shots_contribution = (shots_quality * weight) / total_weight
        
        # 4. Defense Component - based on goals conceded and saves
        # Lower goals conceded is better
        defense_quality = 1 - min(match.get('goals_conceded', 0) / 3, 1)  # Range 0 to 1
        if match.get('goalkeeper_saves', 0) > 0:
            # Adjust for good saves
            defense_quality = min(defense_quality + (match['goalkeeper_saves'] / 10), 1)
        defense_contribution = (defense_quality * weight) / total_weight
        
        # 5. Possession Component - based on possession and passing
        possession_quality = min(match.get('ball_possession', 0) + match.get('pass_accuracy', 0) / 2, 1)
        possession_contribution = (possession_quality * weight) / total_weight
        
        # Store match details
        components["match_details"].append({
            "round": match['round'],
            "opponent_id": match['opponent_id'],
            "opponent_name": match.get('opponent_name', f"Team {match['opponent_id']}"),
            "result": match['result'],
            "score": f"{match['goals_scored']}-{match['goals_conceded']}",
            "weight": weight,
            "results_contribution": results_contribution,
            "goals_contribution": goals_contribution,
            "shots_contribution": shots_contribution,
            "defense_contribution": defense_contribution,
            "possession_contribution": possession_contribution
        })
        
        # Add to component totals
        components["results_component"] += results_contribution
        components["goals_component"] += goals_contribution
        components["shots_component"] += shots_contribution
        components["defense_component"] += defense_contribution
        components["possession_component"] += possession_contribution
    
    # NEW WEIGHTS: 75% for results, 25% distributed equally (6.25% each)
    # Calculate weighted form
    weighted_form = (
        0.70 * components["results_component"] * 5/3 +  # Scale from 0-3 to 0-5
        0.075 * (components["goals_component"] + 1) * 2.5 +  # Scale from -1-1 to 0-5
        0.075 * components["shots_component"] * 5 +  # Scale from 0-1 to 0-5
        0.075 * components["defense_component"] * 5 +  # Scale from 0-1 to 0-5
        0.075 * components["possession_component"] * 5  # Scale from 0-1 to 0-5
    )
    
    # Scale to min-max range
    form_rating = max(min(weighted_form, scale_max), scale_min)
    
    # Store final components
    components["final_form"] = form_rating
    components["results_weighted"] = 0.70 * components["results_component"] * 5/3
    components["goals_weighted"] = 0.075 * (components["goals_component"] + 1) * 2.5
    components["shots_weighted"] = 0.075 * components["shots_component"] * 5
    components["defense_weighted"] = 0.075 * components["defense_component"] * 5
    components["possession_weighted"] = 0.075 * components["possession_component"] * 5
    
    return form_rating, components

def calculate_match_specific_metrics(match):
    """Calculate metrics specific to a single match (not cumulative)"""
    metrics = {}
    
    # Shooting efficiency
    if match.get('goal_attempts') is not None and match.get('goal_attempts') > 0:
        if match.get('shots_on_goal') is not None:
            metrics["target_ratio"] = safe_divide(match.get('shots_on_goal'), match.get('goal_attempts'))
        
        if match.get('goals_scored') is not None:
            metrics["conversion_ratio"] = safe_divide(match.get('goals_scored'), match.get('goal_attempts'))
    
    # Target to goal ratio
    if match.get('shots_on_goal') is not None and match.get('shots_on_goal') > 0 and match.get('goals_scored') is not None:
        metrics["target_to_goal_ratio"] = safe_divide(match.get('goals_scored'), match.get('shots_on_goal'))
    
    # Defensive metrics
    if match.get('tackles_attempted') is not None and match.get('tackles_attempted') > 0 and match.get('tackles_won') is not None:
        metrics["tackle_success_rate"] = safe_divide(match.get('tackles_won'), match.get('tackles_attempted'))
    
    # Calculate defensive action metrics
    defensive_actions = 0
    if match.get('tackles_won') is not None:
        defensive_actions += match.get('tackles_won')
    if match.get('clearances') is not None:
        metrics["clearance_rate"] = match.get('clearances')
        defensive_actions += match.get('clearances')
    if match.get('interceptions') is not None:
        metrics["interception_rate"] = match.get('interceptions')
        defensive_actions += match.get('interceptions')
    
    metrics["defensive_action_rate"] = defensive_actions
    
    # Possession efficiency
    if match.get('ball_possession') is not None and match.get('ball_possession') > 0:
        if match.get('goal_attempts') is not None:
            metrics["possession_efficiency"] = safe_divide(match.get('goal_attempts'), match.get('ball_possession') * 100)
        
        if match.get('goals_scored') is not None:
            metrics["possession_to_goals"] = safe_divide(match.get('goals_scored'), match.get('ball_possession') * 100)
    
    # Passing metrics
    if match.get('passes_attempted') is not None and match.get('passes_attempted') > 0:
        if match.get('passes_completed') is not None:
            metrics["pass_accuracy"] = safe_divide(match.get('passes_completed'), match.get('passes_attempted'))
        
        if match.get('final_third_passes_attempted') is not None and match.get('final_third_passes_attempted') > 0:
            metrics["final_third_entry_ratio"] = safe_divide(match.get('final_third_passes_attempted'), match.get('passes_attempted'))
            
            if match.get('final_third_passes_completed') is not None:
                metrics["final_third_accuracy"] = safe_divide(match.get('final_third_passes_completed'), match.get('final_third_passes_attempted'))
    
    return metrics

def derive_match_stats(team_data, max_rounds=34):
    """
    Calculate derived match statistics from raw match data, including advanced team form
    """
    team_data.sort(key=lambda x: x.get('round', 0))
    regular_matches = [m for m in team_data if m.get('round', 0) <= max_rounds]
    derived_stats = []
    
    # Define all potential KPIs we want to track
    all_potential_kpis = [
        'matches_played',
        'goals_scored',
        'goals_conceded',
        'goal_attempts',
        'shots_on_goal',
        'shots_off_goal',
        'blocked_shots',
        'passes_attempted',
        'passes_completed',
        'final_third_passes_attempted',
        'final_third_passes_completed',
        'tackles_attempted',
        'tackles_won',
        'clearances',
        'interceptions',
        'corner_kicks',
        'fouls',
        'yellow_cards',
        'red_cards'
    ]
    
    # Initialize special field
    all_potential_kpis.append('possession_total')  # We'll calculate this from ball_possession
    
    # Determine which fields are actually present in any match
    available_fields = set()
    for match in regular_matches:
        available_fields.update(match.keys())
    
    # Initialize cumulative trackers
    visualization_cumulative = {'matches_played': 0}
    predictive_cumulative = {'matches_played': 0}
    
    # Initialize all available KPIs
    for key in all_potential_kpis:
        # For standard fields
        if key in available_fields and key != 'possession_total':
            visualization_cumulative[key] = 0
            predictive_cumulative[key] = 0
        # Special handling for possession which might be named differently
        elif key == 'possession_total' and 'ball_possession' in available_fields:
            visualization_cumulative[key] = 0
            predictive_cumulative[key] = 0
    
    # Process each match
    for i, match in enumerate(regular_matches):
        # Create base derived stats object
        derived = {
            "id": f"{match.get('team_id')}_{match.get('match_id')}",
            "team_match_id": match.get('match_id'),
            "round": match.get('round'),
            "date": match.get('date'),
            "opponent_id": match.get('opponent_id'),
            "home_advantage": match.get('is_home'),
        }
        
        # Store the predictive cumulative totals (BEFORE this match)
        derived["predictive_cumulative"] = {k: v for k, v in predictive_cumulative.items()}
        
        # Calculate advanced form for predictive metrics (using data BEFORE this match)
        predictive_metrics = {}
        
        if predictive_cumulative['matches_played'] > 0:
            # Goals metrics
            if 'goals_scored' in predictive_cumulative:
                predictive_metrics["trailing_goals_avg"] = predictive_cumulative['goals_scored'] / predictive_cumulative['matches_played']
            
            if 'goals_conceded' in predictive_cumulative:
                predictive_metrics["trailing_conceded_avg"] = predictive_cumulative['goals_conceded'] / predictive_cumulative['matches_played']
            
            # Shooting metrics
            if 'goal_attempts' in predictive_cumulative:
                predictive_metrics["trailing_shots_avg"] = predictive_cumulative['goal_attempts'] / predictive_cumulative['matches_played']
            
            if 'shots_on_goal' in predictive_cumulative:
                predictive_metrics["trailing_shots_on_target_avg"] = predictive_cumulative['shots_on_goal'] / predictive_cumulative['matches_played']
            
            # Passing metrics
            if 'passes_completed' in predictive_cumulative and 'passes_attempted' in predictive_cumulative and predictive_cumulative['passes_attempted'] > 0:
                predictive_metrics["trailing_pass_accuracy"] = predictive_cumulative['passes_completed'] / predictive_cumulative['passes_attempted']
            
            # Possession metrics
            if 'possession_total' in predictive_cumulative:
                predictive_metrics["trailing_possession_avg"] = predictive_cumulative['possession_total'] / predictive_cumulative['matches_played']
            
            # Defensive metrics
            if 'tackles_won' in predictive_cumulative and 'tackles_attempted' in predictive_cumulative and predictive_cumulative['tackles_attempted'] > 0:
                predictive_metrics["trailing_tackle_success"] = predictive_cumulative['tackles_won'] / predictive_cumulative['tackles_attempted']
            
            defensive_actions = 0
            if 'tackles_won' in predictive_cumulative:
                defensive_actions += predictive_cumulative['tackles_won']
            if 'clearances' in predictive_cumulative:
                defensive_actions += predictive_cumulative['clearances']
            if 'interceptions' in predictive_cumulative:
                defensive_actions += predictive_cumulative['interceptions']
            
            if defensive_actions > 0:
                predictive_metrics["trailing_defensive_actions_avg"] = defensive_actions / predictive_cumulative['matches_played']
            
            # Calculate advanced team form (using matches leading up to current round)
            if i > 0:  # Only if there are previous matches
                prev_matches = regular_matches[:i]
                form_rating, form_components = calculate_team_form(prev_matches, scale_min=1, scale_max=5)
                predictive_metrics["team_form"] = form_rating
                predictive_metrics["team_form_components"] = {
                    "results_component": form_components["results_weighted"],
                    "goals_component": form_components["goals_weighted"],
                    "shots_component": form_components["shots_weighted"],
                    "defense_component": form_components["defense_weighted"],
                    "possession_component": form_components["possession_weighted"]
                }
            else:
                predictive_metrics["team_form"] = 3  # Neutral form if no prior matches
        
        derived["predictive"] = predictive_metrics
        
        # Now update running totals WITH current match for visualization
        visualization_cumulative['matches_played'] += 1
        
        # Update standard statistics
        for key in visualization_cumulative:
            if key in match and match[key] is not None and key not in ['matches_played', 'possession_total']:
                visualization_cumulative[key] += match[key]
        
        # Special handling for possession
        if 'ball_possession' in match and match['ball_possession'] is not None:
            visualization_cumulative['possession_total'] += match['ball_possession']
        
        # Store the visualization cumulative totals (INCLUDING this match)
        derived["visualization_cumulative"] = {k: v for k, v in visualization_cumulative.items()}
        
        # Calculate visualization metrics (including current match)
        visualization_metrics = {}
        
        if visualization_cumulative['matches_played'] > 0:
            # Goals metrics
            if 'goals_scored' in visualization_cumulative:
                visualization_metrics["goals_rolling_avg"] = visualization_cumulative['goals_scored'] / visualization_cumulative['matches_played']
            
            if 'goals_conceded' in visualization_cumulative:
                visualization_metrics["conceded_goals_rolling_avg"] = visualization_cumulative['goals_conceded'] / visualization_cumulative['matches_played']
            
            # Shooting metrics
            if 'goal_attempts' in visualization_cumulative:
                visualization_metrics["shots_rolling_avg"] = visualization_cumulative['goal_attempts'] / visualization_cumulative['matches_played']
            
            if 'shots_on_goal' in visualization_cumulative:
                visualization_metrics["shots_on_goal_rolling_avg"] = visualization_cumulative['shots_on_goal'] / visualization_cumulative['matches_played']
            
            # Calculate ratios and efficiencies
            if 'shots_on_goal' in visualization_cumulative and 'goal_attempts' in visualization_cumulative and visualization_cumulative['goal_attempts'] > 0:
                visualization_metrics["shot_accuracy"] = visualization_cumulative['shots_on_goal'] / visualization_cumulative['goal_attempts']
            
            if 'goals_scored' in visualization_cumulative and 'shots_on_goal' in visualization_cumulative and visualization_cumulative['shots_on_goal'] > 0:
                visualization_metrics["conversion_rate"] = visualization_cumulative['goals_scored'] / visualization_cumulative['shots_on_goal']
            
            # Possession metrics
            if 'possession_total' in visualization_cumulative:
                visualization_metrics["possession_avg"] = visualization_cumulative['possession_total'] / visualization_cumulative['matches_played']
            
            # Defensive metrics
            defensive_actions = 0
            if 'tackles_won' in visualization_cumulative:
                defensive_actions += visualization_cumulative['tackles_won']
            if 'clearances' in visualization_cumulative:
                defensive_actions += visualization_cumulative['clearances']
            if 'interceptions' in visualization_cumulative:
                defensive_actions += visualization_cumulative['interceptions']
            
            if defensive_actions > 0:
                visualization_metrics["defensive_actions_avg"] = defensive_actions / visualization_cumulative['matches_played']
            
            # Calculate advanced team form (including current match)
            form_rating, form_components = calculate_team_form(regular_matches[:i+1], scale_min=1, scale_max=5)
            visualization_metrics["team_form"] = form_rating
            visualization_metrics["team_form_components"] = {
                "results_component": form_components["results_weighted"],
                "goals_component": form_components["goals_weighted"],
                "shots_component": form_components["shots_weighted"],
                "defense_component": form_components["defense_weighted"],
                "possession_component": form_components["possession_weighted"]
            }
        
        derived["visualization"] = visualization_metrics
        
        # Add match-specific metrics
        derived["match_metrics"] = calculate_match_specific_metrics(match)
        
        # Add to derived stats collection
        derived_stats.append(derived)
        
        # AFTER processing the match, update predictive totals for next match
        predictive_cumulative['matches_played'] += 1
        
        # Update all other metrics
        for key in predictive_cumulative:
            if key in match and match[key] is not None and key not in ['matches_played', 'possession_total']:
                predictive_cumulative[key] += match[key]
        
        # Special handling for possession
        if 'ball_possession' in match and match['ball_possession'] is not None:
            predictive_cumulative['possession_total'] += match['ball_possession']
    
    return derived_stats

In [51]:
import os
import json 
import pandas as pd

def delete_all_derived_stats(base_dir):
    """Delete all derived_stats.json files in the directory structure"""
    count = 0
    
    print(f"Searching for derived_stats.json files in {base_dir}...")
    
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file == "derived_stats.json":
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    count += 1
                    print(f"  Deleted: {file_path}")
                except Exception as e:
                    print(f"  Error deleting {file_path}: {str(e)}")
    
    print(f"Deleted {count} derived_stats.json files")
    return count

# Main script to create derived_stats.json files
def process_teams_data(curr_league, delete_existing=True):
    base_dir = f"../data/leagues/{curr_league}"
    
    # Delete existing derived_stats.json files if requested
    if delete_existing:
        delete_all_derived_stats(base_dir)
    
    # First, lets get the id mappings from info.json
    with open(base_dir+"/info.json", 'r') as f:
        league_info = json.load(f)
    
    # get the ids
    team_id_mappings = {team_key: team_data['id'] for team_key, team_data in league_info['teams'].items()}
    
    # Find all seasons in our base directory
    seasons = sorted([season for season in os.listdir(base_dir) if os.path.isdir(f'{base_dir}/{season}') and season != "rounds"])
    
    print(f"Processing {len(seasons)} seasons for {curr_league}")
    
    for season in seasons:
        print(f"\nProcessing season: {season}")
        
        # Each season has a teams directory
        season_teams_dir = f"{base_dir}/{season}/teams"
        
        # Skip if teams directory doesn't exist
        if not os.path.exists(season_teams_dir):
            print(f"  No teams directory found for {season}, skipping")
            continue
        
        # Get list of team directories
        team_dirs = [d for d in os.listdir(season_teams_dir) if os.path.isdir(f'{season_teams_dir}/{d}')]
        
        print(f"  Found {len(team_dirs)} teams")
        
        for team_key in team_dirs:
            team_dir = f"{season_teams_dir}/{team_key}"
            match_stats_file = f"{team_dir}/match_stats.json"
            derived_stats_file = f"{team_dir}/derived_stats.json"
            
            # Skip if match_stats.json doesn't exist
            if not os.path.exists(match_stats_file):
                print(f"  No match_stats.json found for {team_key}, skipping")
                continue
            
            try:
                # Load team's match data
                with open(match_stats_file, 'r') as f:
                    team_data = json.load(f)
                
                print(f"  Processing {team_key} with {len(team_data)} matches")
                
                # Generate derived stats
                derived_stats = derive_match_stats(team_data)
                
                # Save derived stats
                with open(derived_stats_file, 'w') as f:
                    json.dump(derived_stats, f, indent=2)
                
                print(f"  Created derived_stats.json for {team_key}")
                
            except Exception as e:
                print(f"  Error processing {team_key}: {str(e)}")
    
    print("\nAll teams processed!")

# Run the script
if __name__ == "__main__":
    # Set to True to delete all existing derived_stats.json files first
    process_teams_data("serie_a", delete_existing=False)

Processing 8 seasons for serie_a

Processing season: 2017_2018
  Found 20 teams
  Processing spal with 30 matches
  Error processing spal: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing ac_milan with 30 matches
  Error processing ac_milan: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing atalanta with 30 matches
  Error processing atalanta: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing torino with 30 matches
  Error processing torino: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing bologna with 30 matches
  Error processing bologna: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing lazio with 30 matches
  Error processing lazio: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing sassuolo with 30 matches
  Error processing sassuolo: unsupported operand type(s) for /: 'NoneType' and 'int'
  Processing crotone with 29 matches
  Error processing crotone: unsupported op

In [52]:
import json
import os
import pandas as pd
import numpy as np
from collections import defaultdict

def calculate_team_season_metrics(base_dir, curr_league):
    """Calculate and save team season metrics for all teams across all seasons."""
    
    # Get all seasons
    seasons_path = f"{base_dir}/seasons.json"
    with open(seasons_path, 'r') as f:
        seasons_data = json.load(f)
    
    # Process each season
    for season_key, season_info in seasons_data.items():
        season_dir = f"{base_dir}/{season_key}"
        
        # Skip if not a directory or doesn't exist
        if not os.path.isdir(season_dir):
            continue
        
        teams_dir = f"{season_dir}/teams"
        if not os.path.exists(teams_dir):
            continue
            
        # Process each team in this season
        for team_key in os.listdir(teams_dir):
            team_dir = f"{teams_dir}/{team_key}"
            
            # Skip if not a directory
            if not os.path.isdir(team_dir):
                continue
                
            # Check for match_stats.json
            match_stats_path = f"{team_dir}/match_stats.json"
            if not os.path.exists(match_stats_path):
                continue
                
            # Load the team's match stats
            with open(match_stats_path, 'r') as f:
                match_stats = json.load(f)
                
            # Calculate season metrics
            season_metrics = calculate_metrics(match_stats, team_key, season_key, season_info['id'])
            
            # Save season totals
            with open(f"{team_dir}/season_totals.json", 'w') as f:
                json.dump(season_metrics, f, indent=4)
            
            print(f"Saved season totals for {team_key} in {season_key}")

def calculate_metrics(match_stats, team_key, season_key, season_id):
    """Calculate aggregated season metrics from a team's match stats."""
    
    # Initialize basic season metrics
    metrics = {
        "team_id": match_stats[0]["team_id"] if match_stats else None,
        "team_key": team_key,
        "team_name": match_stats[0]["team_name"] if match_stats else None,
        "season_id": season_id,
        "season": season_key,
        "matches_played": len(match_stats),
        "wins": 0,
        "draws": 0,
        "losses": 0,
        "points": 0,
        "goals_for": 0,
        "goals_against": 0,
        "goal_difference": 0,
        "clean_sheets": 0,
        "failed_to_score": 0,
        "home_matches": 0,
        "away_matches": 0,
        "home_wins": 0,
        "home_draws": 0,
        "home_losses": 0,
        "away_wins": 0,
        "away_draws": 0,
        "away_losses": 0,
        "home_goals_for": 0,
        "home_goals_against": 0,
        "away_goals_for": 0,
        "away_goals_against": 0,
        "form": []  # Recent results (W, D, L)
    }
    
    # Aggregated statistics
    total_stats = defaultdict(int)
    count_stats = defaultdict(int)
    min_stats = defaultdict(lambda: float('inf'))
    max_stats = defaultdict(lambda: float('-inf'))
    
    # Process each match
    for match in match_stats:
        # Basic match outcome
        result = match["result"]
        if result == "win":
            metrics["wins"] += 1
            if match["is_home"]:
                metrics["home_wins"] += 1
            else:
                metrics["away_wins"] += 1
        elif result == "draw":
            metrics["draws"] += 1
            if match["is_home"]:
                metrics["home_draws"] += 1
            else:
                metrics["away_draws"] += 1
        else:  # loss
            metrics["losses"] += 1
            if match["is_home"]:
                metrics["home_losses"] += 1
            else:
                metrics["away_losses"] += 1
        
        # Points, goals, etc.
        metrics["points"] += match["points"]
        metrics["goals_for"] += match["goals_scored"]
        metrics["goals_against"] += match["goals_conceded"]
        
        if match["is_home"]:
            metrics["home_matches"] += 1
            metrics["home_goals_for"] += match["goals_scored"]
            metrics["home_goals_against"] += match["goals_conceded"]
        else:
            metrics["away_matches"] += 1
            metrics["away_goals_for"] += match["goals_scored"]
            metrics["away_goals_against"] += match["goals_conceded"]
        
        # Clean sheets and failures to score
        if match["goals_conceded"] == 0:
            metrics["clean_sheets"] += 1
        if match["goals_scored"] == 0:
            metrics["failed_to_score"] += 1
        
        # Calculate form (last 5 matches, most recent first)
        if len(metrics["form"]) < 5:
            metrics["form"].insert(0, result[0].upper())  # W, D, or L
        
        # Process all numeric statistics
        for key, value in match.items():
            # Skip non-numeric or already processed values
            if key in ["team_id", "team_key", "team_name", "opponent_id", "opponent_key", 
                     "opponent_name", "result", "date", "is_home", "round", "match_id",
                     "season_id", "season", "points", "goals_scored", "goals_conceded"] or value is None:
                continue
            
            # Skip string or non-numeric values
            if not isinstance(value, (int, float)) or isinstance(value, bool):
                continue
                
            # For each numeric stat, track total, count, min and max
            total_stats[key] += value
            count_stats[key] += 1
            min_stats[key] = min(min_stats[key], value)
            max_stats[key] = max(max_stats[key], value)
    
    # Calculate averages, mins, and maxes for all tracked stats
    metrics["averages"] = {}
    metrics["mins"] = {}
    metrics["maxes"] = {}
    
    for key in total_stats:
        if count_stats[key] > 0:
            metrics["averages"][key] = total_stats[key] / count_stats[key]
            metrics["mins"][key] = min_stats[key]
            metrics["maxes"][key] = max_stats[key]
    
    # Calculate final calculated metrics
    metrics["goal_difference"] = metrics["goals_for"] - metrics["goals_against"]
    metrics["points_per_game"] = metrics["points"] / metrics["matches_played"] if metrics["matches_played"] > 0 else 0
    metrics["win_percentage"] = (metrics["wins"] / metrics["matches_played"]) * 100 if metrics["matches_played"] > 0 else 0
    metrics["home_win_percentage"] = (metrics["home_wins"] / metrics["home_matches"]) * 100 if metrics["home_matches"] > 0 else 0
    metrics["away_win_percentage"] = (metrics["away_wins"] / metrics["away_matches"]) * 100 if metrics["away_matches"] > 0 else 0
    
    # Add more advanced metrics if data is available
    if "shot_accuracy" in metrics["averages"]:
        metrics["shot_accuracy_season"] = metrics["averages"]["shot_accuracy"]
    if "pass_accuracy" in metrics["averages"]:
        metrics["pass_accuracy_season"] = metrics["averages"]["pass_accuracy"]
    if "possession" in metrics["averages"]:
        metrics["possession_average"] = metrics["averages"]["possession"]
    
    # Converting lists to strings for better JSON serialization
    metrics["form"] = "".join(metrics["form"])
    
    return metrics

# Usage
base_dir = "../data/leagues/serie_a"
curr_league = "serie_a"
calculate_team_season_metrics(base_dir, curr_league)

Saved season totals for spal in 2017_2018
Saved season totals for ac_milan in 2017_2018
Saved season totals for atalanta in 2017_2018
Saved season totals for torino in 2017_2018
Saved season totals for bologna in 2017_2018
Saved season totals for lazio in 2017_2018
Saved season totals for sassuolo in 2017_2018
Saved season totals for crotone in 2017_2018
Saved season totals for as_roma in 2017_2018
Saved season totals for cagliari in 2017_2018
Saved season totals for udinese in 2017_2018
Saved season totals for juventus in 2017_2018
Saved season totals for inter in 2017_2018
Saved season totals for fiorentina in 2017_2018
Saved season totals for genoa in 2017_2018
Saved season totals for verona in 2017_2018
Saved season totals for benevento in 2017_2018
Saved season totals for sampdoria in 2017_2018
Saved season totals for chievo in 2017_2018
Saved season totals for napoli in 2017_2018
Saved season totals for spal in 2018_2019
Saved season totals for ac_milan in 2018_2019
Saved season 